***
## Normalizacion de las tabas de referencia REPS, criterios sobre la data:
- Fue necesario normalizar la tablas dispuestas en https://prestadores.minsalud.gov.co/habilitacion/ 
    - (Los CSV, tienen inconsistencias en sus campos, no correspondencia con los encabezados)
- Llaves de homologacion para con la tabla Municipios 
    * Se toma en cuenta los campos: 
        * codigo_habilitacion

In [61]:
import pandas as pd
import sqlite3

In [62]:
# Forzar el engine 'xlrd' (necesitas tener xlrd instalado)
TABLASREPS = pd.read_excel("01_PRUEBA_IETS\\01_INSUMO_REPS_Vigente\\00_Consolidado_REPS.xlsx", sheet_name=None)
TABLASREPS.keys()

dict_keys(['01_Prestadores', '02_Servicios', '03_Sedes', '04_CapacidadInstalada', '05_MedidasSeguridad', '06_Sanciones'])

***
### Validacion de la integridad referencial de los codigo_habilitacion, de las tablas REPS.

 - Se extrael el codigo de municipio a partir del *codigo_habilitacion* para hacer el merge, ejemplo: *9977300145*  →  <span style="color: #ccff99;"><b>99773</b></span> | *00145*

 - Se validan las referencias contenidas en la tabla Municipios
 - Se valida el *codigo_habilitacion* de todas las tablas reportadas en REPS vigente:

    - <span style="color:#1f77b4;"><b>01_Prestadores.csv:</b></span> Contiene datos identificativos y administrativos de los prestadores de servicios de salud registrados, incluyendo códigos, nombres, tipo de entidad y estado de habilitación.

    - <span style="color:#ff7f0e;"><b>02_Servicios.csv:</b></span> Detalla los servicios de salud ofrecidos por los prestadores, con información de códigos, nombres, clasificación y modalidades de atención.

    - <span style="color:#2ca02c;"><b>03_Sedes.csv:</b></span> Registra las sedes físicas de los prestadores, indicando ubicación, dirección, códigos geográficos y datos de contacto.

    - <span style="color:#d62728;"><b>04_CapacidadInstalada.csv:</b></span> Describe la capacidad instalada de las sedes, con información sobre número de camas, unidades de servicio y equipamiento disponible.

    - <span style="color:#9467bd;"><b>05_MedidasSeguridad.csv:</b></span> Incluye las medidas de seguridad implementadas por los prestadores para garantizar la calidad y seguridad del servicio.

    - <span style="color:#8c564b;"><b>06_Sanciones.csv:</b></span> Registra las sanciones administrativas impuestas a prestadores o sedes, con detalles sobre motivos, fechas y estado de los procesos.



In [68]:
codigo_habilitacion = []  # 
for TAB_REPS in TABLASREPS.keys():
    codigo_habilitacion += TABLASREPS[TAB_REPS]['codigo_habilitacion'].to_list() # Recorrido por todas las hojas del libro REPS vigente

In [69]:
def quitar_ultimos_5_digitos(numero): # Funcion para obtener el campo "MPIO", de la tabla Municipios
    return int(str(numero)[:-5])

DF_CODHABI = pd.DataFrame(set(codigo_habilitacion), columns = ['codigo_habilitacion'])  
DF_CODHABI['MPIO'] = DF_CODHABI['codigo_habilitacion'].apply(quitar_ultimos_5_digitos)  # Generacion del codigo MPIO
DF_CODHABI.head(4)
DF_CODHABI['VALID'] = "SI"

***
### Cruce con la tabla Municipios (Normalizada)

* Se valida integridad referencial de los codigos 'MPIO', contenidos en las tablas REPS viegente
*  <strong style="color:red;">Los siguientes municipios tienen alguna novedad con el código MPIO, No se encontró referencia en las tabla REPS vigente</strong>








In [70]:
MUNICIPIOS = pd.read_excel("02_Tablas_Normalizadas\\02_Municipios Normalizado.xlsx")
MUNICIPIOS2 = MUNICIPIOS.merge(DF_CODHABI, how = "left")
MUNICIPIOS2['VALID'] = MUNICIPIOS2['VALID'].fillna("NO")  # El campo VALID se utiliza para validar integrida referencial, iden
MUNICIPIOS2[MUNICIPIOS2['VALID'] == "NO"][['DP', 'Departamento', 'MPIO', 'Municipio']]

,DP,Departamento,MPIO,Municipio
64955,94,Guainía,94663,Mapiripana
64957,94,Guainía,94884,Puerto Colombia
64958,94,Guainía,94885,La Guadalupe
64959,94,Guainía,94886,Cacahual
64960,94,Guainía,94887,Pana Pana
64961,94,Guainía,94888,Morichal


***
## Generacion de las tablas normalizadas
- Se hace el cruce de las Tablas REPS_Vigente con la tabla Municipios aportada 
- Se crea la tabla relacional utilizando el motor de SQLite.
- La base de datos ba a ser una que contiene las siguientes tablas:
    - '01_Prestadores'
    - '02_Servicios'
    - '03_Sedes'
    - '04_CapacidadInstalada'
    - '05_MedidasSeguridad'
    - '06_Sanciones'


In [ ]:
CAMPOS_Municipios  = ['DP',
                      #'Departamento', # Se opiten estos campos en la creacion de la base de datos SQLite, para evitar error
                      'MPIO',
                      #'Municipio',    # Se opiten estos campos en la creacion de la base de datos SQLite, para evitar error
                      'Superficie',
                      'PopTot',
                      'Rural',
                      'Region']

In [ ]:
conn = sqlite3.connect("02_Tablas_Normalizadas\\reps_database.db")

for Tabla_REPS in TABLASREPS.keys():
    DFPROV = TABLASREPS[Tabla_REPS]
    DFPROV['MPIO'] = DFPROV['codigo_habilitacion'].apply(quitar_ultimos_5_digitos)
    DFPROV = DFPROV.merge(MUNICIPIOS[CAMPOS_Municipios], how="inner", left_on='MPIO', right_on='MPIO')
    DFPROV = DFPROV.loc[:,~DFPROV.columns.duplicated()]   # Se eliminan columnas duplicadas

    DFPROV.to_sql(Tabla_REPS, conn, if_exists="replace", index=False) # Se agregan las tablas en la base de datos reps_database.db
conn.close()

In [84]:
import sqlite3
import pandas as pd

def describir_tablas_sqlite(ruta_db):
    conn = sqlite3.connect(ruta_db)
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tablas = [t[0] for t in cursor.fetchall()]

    for tabla in tablas:
        print(f"\nTabla: {tabla}")

        # Número de filas
        cursor.execute(f'SELECT COUNT(*) FROM "{tabla}";')
        num_filas = cursor.fetchone()[0]
        print(f"Número de filas: {num_filas}")

        # Columnas y tipos
        cursor.execute(f'PRAGMA table_info("{tabla}");')
        columnas = cursor.fetchall()  # (cid, name, type, notnull, dflt_value, pk)
        num_campos = len(columnas)
        print(f"Número de campos: {num_campos}")

        nombres_campos = [col[1] for col in columnas]
        print(f"Campos principales: {', '.join(nombres_campos)}")

    conn.close()
# Uso
ruta_base_datos = "02_Tablas_Normalizadas\\reps_database.db"
describir_tablas_sqlite(ruta_base_datos)



Tabla: 01_Prestadores
Número de filas: 59351
Número de campos: 43
Campos principales: depa_nombre, muni_nombre, codigo_habilitacion, nombre_prestador, tido_codigo, nits_nit, razon_social, clpr_codigo, clpr_nombre, ese, direccion, telefono, fax, email, gerente, nivel, caracter, habilitado, fecha_radicacion, fecha_vencimiento, fecha_cierre, dv, clase_persona, naju_codigo, naju_nombre, numero_sede_principal, fecha_corte_REPS, telefono_adicional, email_adicional, rep_legal, Municipio PDET, Municipio ZOMAC, Municipio PNIS, Municipio PNSR antes 2023, Municipio PNSR 2023, Municipio PNSR 2024, Column1, MPIO, DP, Superficie, PopTot, Rural, Region

Tabla: 02_Servicios
Número de filas: 227856
Número de campos: 100
Campos principales: depa_nombre, muni_nombre, habi_codigo_habilitacion, codigo_habilitacion, numero_sede, sede_nombre, direccion, telefono, email, nits_nit, dv, clase_persona, naju_codigo, naju_nombre, clpr_codigo, clpr_nombre, ese, nivel, caracter, habilitado, grse_codigo, grse_nombre